# HW05: Double ML with XGBoost
(due October 27th)

By: Ulrich Bergmann

- I have a problem with the XGBoost fit below. Please let me know where my mistake lies.

In [2]:
# Load Data (NLSY)
import pandas as pd
df = pd.read_stata('http://www.stata-press.com/data/r10/nlswork.dta')

In [3]:
import numpy as np
Y = df['ln_wage']
D = df['union']

In [4]:
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28534 entries, 0 to 28533
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   idcode    28534 non-null  int16  
 1   year      28534 non-null  int8   
 2   birth_yr  28534 non-null  int8   
 3   age       28510 non-null  float64
 4   race      28534 non-null  int8   
 5   msp       28518 non-null  float64
 6   nev_mar   28518 non-null  float64
 7   grade     28532 non-null  float64
 8   collgrad  28534 non-null  int8   
 9   not_smsa  28526 non-null  float64
 10  c_city    28526 non-null  float64
 11  south     28526 non-null  float64
 12  ind_code  28193 non-null  float64
 13  occ_code  28413 non-null  float64
 14  union     19238 non-null  float64
 15  wks_ue    22830 non-null  float64
 16  ttl_exp   28534 non-null  float32
 17  tenure    28101 non-null  float32
 18  hours     28467 non-null  float64
 19  wks_work  27831 non-null  float64
 20  ln_wage   28534 non-null  fl

In [5]:
# fill in X with all predictors that are not colliders
X = df[['age', 'year', 'race', 'msp', 'nev_mar', 'grade', 'collgrad', 'c_city', 'south', 'wks_ue', 'ttl_exp', 'tenure', 'wks_work' ]]

In [6]:
# split into sample A and sample B
Ax = X.sample(frac = 0.5)
Ay = Y[Ax.index]
Ad = D[Ax.index]

Bx = X.drop(Ax.index)
By = Y.drop(Ax.index)
Bd = D.drop(Ax.index)
  
# Within each sample, make a validation set for xgboost early stopping

from sklearn.model_selection import train_test_split 
Ax_train, Ax_val, Ay_train, Ay_val = train_test_split(Ax, Ay)
Bx_train, Bx_val, By_train, By_val = train_test_split(Bx, By)

Ad_train = Ad[Ax_train.index]
Ad_val = Ad[Ax_val.index]

Bd_train = Bd[Bx_train.index]
Bd_val = Bd[Bx_val.index]

### The fit in the next cell takes forever. I'm not sure if I screwed up the index  in the previous cell?

HELP :D


In [ ]:
# Step 1. In both samples, use xgboost regressor to predict log wages (outcome Y)
# use early stopping.
from xgboost import XGBClassifier
AY_model = XGBClassifier(eval_set = [Ax_val, Ay_val], early_stopping_rounds=10)
AY_model.fit(Ax_train, Ay_train, verbose=True)

BY_model = XGBClassifier(eval_set = [Bx_val, By_val], early_stopping_rounds=10)
BY_model.fit(Bx_train, By_train, verbose=True)

In [ ]:
# Step 2. In both samples, use xgboost classifier to predict union status (treatment D)
# use early stopping.
from xgboost import XGBClassifier
AD_model = XGBClassifier(eval_set = [Ax_val, Ad_val], early_stopping_rounds=10)
AD_model.fit(Ax_train, Ad_train)

BD_model = XGBClassifier(eval_set = [Bx_val, Bd_val], early_stopping_rounds=10)
BD_model.fit(Bx_train, Bd_train)

In [ ]:
# Step 3. Cross-fitting: Form predictions in other sample.

# predict wages in sample A using model trained in sample B:
Y_hat_A = BY_model.predict(Ax)

# vice versa:
Y_hat_B = AY_model.predict(Bx)
    
# predict union status in sample A using model trained in sample B:
D_hat_A = BD_model.predict(Ax)
    
# vice versa:
D_hat_B = AD_model.predict(Bx)
    

In [ ]:
# Compute residuals for wages and union status.

# residualized wages in samples A and B:
Y_tilde_A =  Ay - Y_hat_A
Y_tilde_B =  By - Y_hat_B

# residualized union status in samples A and B:
D_tilde_A =  Ad - D_hat_A
D_tilde_B =  Bd - D_hat_B

In [12]:
# Step 4. Run OLS regressions and produce DML estimate

import statsmodels.api as sm

model_A = sm.OLS(Y_tilde_A, D_tilde_A)
result_A = model.fit(cov_type='HC1')
beta_A = result.params
se_A = result.se

model_B = sm.OLS(Y_tilde_B, D_tilde_B)
result_B = model.fit(cov_type='HC1')
beta_B = result.params
se_B = result.se

beta = .5 * (beta_A + beta_B)
se = .5 * (se_A + se_B)

print('DML Coeff:',beta)
print('DML S.E.:', se)

NameError: name 'Y_tilde_A' is not defined